In [1]:
import os
import pandas as pd
import numpy as np
from transformers import BertTokenizer
import torch

import sys
sys.path.append('/home/nauel/bert_gender_bias')
import warnings
warnings.filterwarnings("ignore")


from pipelines.utils.paths import EXTERNAL_DATA_DIR, INTERIM_DATA_DIR

In [2]:
gendered_words = pd.read_pickle(os.path.join(INTERIM_DATA_DIR, 'gender_binary_words_TOKEN.pkl'))

In [3]:
gendered_words.dropna(inplace=True)
print(gendered_words.shape)
gendered_words.head(1000)

(431, 5)


,word,gender_binary,bert_token,word2vec_token,word_count
0,abbot,0,"[-0.39571184, -0.093838364, 0.06868138, 0.1723...","[0.40039062, 0.41015625, 0.36523438, 0.2207031...",1
1,abbots,0,"[-0.18790531, -0.077521764, -0.3876859, 0.1816...","[0.30664062, 0.33398438, 0.1953125, 0.50390625...",1
2,adulterer,0,"[-0.45352724, -0.38397712, -0.25277817, -0.266...","[0.42773438, -0.28515625, -0.0625, 0.020263672...",1
3,adulterers,0,"[-0.17249976, -0.15101261, -0.3717648, -0.0279...","[-0.14160156, -0.22558594, -0.042236328, 0.208...",1
4,airman,0,"[-0.20044291, -0.020562049, -0.276353, -0.0372...","[0.49023438, -0.15917969, -0.022460938, -0.024...",1
...,...,...,...,...,...
510,woman,1,"[-0.34769166, -0.20875905, 0.12518345, 0.07620...","[0.24316406, -0.07714844, -0.103027344, -0.107...",1
511,womankind,1,"[-0.24836135, -0.283562, -0.19828826, 0.071805...","[-0.096191406, -0.055908203, 0.014404297, 0.22...",1
512,womanly,1,"[-0.3582225, -0.31381592, -0.16148914, 0.21661...","[0.01965332, -0.059570312, 0.119628906, 0.1474...",1
513,womanpower,1,"[-0.042981803, -0.32953677, 0.07738724, 0.1452...","[-0.0041503906, -0.06225586, 0.044921875, 0.13...",1


In [58]:
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

token_col = 'bert_token'
X = gendered_words[token_col].values.tolist()
y = np.array(gendered_words.gender_binary.tolist())
logit = LogisticRegression(solver="liblinear", penalty="l1", C=1)
logit.fit(X, y)
print("Accuracy: ", logit.score(X, y))
print("Number of non-zero coefficients: ", np.sum(logit.coef_ != 0))
print("Number of zero coefficients: ", np.sum(logit.coef_ == 0))
print("Number of coefficients: ", len(logit.coef_[0]))

Accuracy:  0.9907192575406032
Number of non-zero coefficients:  77
Number of zero coefficients:  691
Number of coefficients:  768


In [175]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
import numpy as np

# Assuming gendered_words DataFrame is already defined

# Preparing the data
token_col = 'bert_token'
X = gendered_words[token_col].values.tolist()
y = np.array(gendered_words['gender_binary'].tolist())

# Defining the model
logit = LogisticRegression(solver="liblinear", penalty="l1")

# Setting up the parameter grid to search for the best C value
param_grid = {'C': np.logspace(-5, 1, 5)}  # You can adjust this range as needed

# Performing grid search with cross-validation
grid_search = GridSearchCV(logit, param_grid, cv=5, scoring='accuracy')
grid_search.fit(X, y)

# Extracting the best model
best_logit = grid_search.best_estimator_

# Print results
print("Best C parameter: ", grid_search.best_params_['C'])
print("Best cross-validated accuracy: ", grid_search.best_score_)
print("Number of non-zero coefficients in the best model: ", np.sum(best_logit.coef_ != 0))
print("Number of zero coefficients in the best model: ", np.sum(best_logit.coef_ == 0))
print("Number of coefficients in the best model: ", len(best_logit.coef_[0]))
print("Number of coeff > 0.1 in the best model: ", np.sum(best_logit.coef_[0] > 0.1))


Best C parameter:  10.0
Best cross-validated accuracy:  0.884095161721465
Number of non-zero coefficients in the best model:  109
Number of zero coefficients in the best model:  659
Number of coefficients in the best model:  768
Number of coeff > 0.1 in the best model:  47


In [176]:
# Convert numpy array to pandas Series
coef_series = pd.Series(best_logit.coef_[0])

# Sort the values in descending order
sorted_coefs = coef_series.sort_values(ascending=False)

sorted_coefs[:50]


521    5.905018
586    4.136882
138    3.534407
52     3.521438
286    3.399265
592    3.282370
640    2.940193
474    2.931977
591    2.632508
741    2.510618
141    2.298457
251    2.216687
394    1.945827
756    1.941335
492    1.738220
237    1.723409
706    1.460800
375    1.420564
481    1.381334
323    1.381124
711    1.368513
409    1.353401
600    1.338773
435    1.331235
33     1.319314
117    1.253673
562    1.211033
342    1.181413
370    1.175977
258    1.152920
622    1.119039
723    1.103538
17     0.999049
218    0.975939
274    0.973168
250    0.904471
464    0.736669
9      0.711074
631    0.633363
571    0.366555
308    0.364685
646    0.358388
540    0.347036
228    0.227692
232    0.212833
653    0.165895
689    0.159256
73     0.013582
505    0.000000
547    0.000000
dtype: float64

In [177]:
# Get the coefficients
coefficients = best_logit.coef_[0]
coefficients_abs = np.abs(coefficients)

logit_coef = pd.DataFrame({'feature': range(len(coefficients)), 'coefficient': coefficients, 'absolute_coefficient': coefficients_abs})
logit_coef = logit_coef.sort_values(by='absolute_coefficient', ascending=False)
logit_coef = logit_coef[logit_coef.absolute_coefficient > 0]
logit_coef.head(200)


,feature,coefficient,absolute_coefficient
521,521,5.905018,5.905018
417,417,-5.193013,5.193013
636,636,-4.864148,4.864148
104,104,-4.205046,4.205046
586,586,4.136882,4.136882
...,...,...,...
105,105,-0.155106,0.155106
518,518,-0.154345,0.154345
708,708,-0.142034,0.142034
164,164,-0.104980,0.104980


In [178]:
selected_features = logit_coef.feature.values

gendered_words['selected_tokens'] = gendered_words.bert_token.apply(lambda x: [x[i] for i in selected_features])
gendered_words.head()

,word,gender_binary,bert_token,word2vec_token,word_count,selected_tokens,pc1,pc2,pc1_norm,pc2_norm,cluster,svm_cluster
0,abbot,0,"[-0.39571184, -0.093838364, 0.06868138, 0.1723...","[0.40039062, 0.41015625, 0.36523438, 0.2207031...",1,"[-0.29727972, 0.13706006, 0.25577983, -0.35511...",-0.585686,0.248053,0.092853,0.590409,0,0
1,abbots,0,"[-0.18790531, -0.077521764, -0.3876859, 0.1816...","[0.30664062, 0.33398438, 0.1953125, 0.50390625...",1,"[-0.09587786, 0.34578195, 0.050730024, -0.1052...",0.427147,0.198370,0.409206,0.568188,1,0
2,adulterer,0,"[-0.45352724, -0.38397712, -0.25277817, -0.266...","[0.42773438, -0.28515625, -0.0625, 0.020263672...",1,"[0.026859095, -0.04974738, -0.00580548, -0.274...",0.603189,0.383462,0.464192,0.650970,1,0
3,adulterers,0,"[-0.17249976, -0.15101261, -0.3717648, -0.0279...","[-0.14160156, -0.22558594, -0.042236328, 0.208...",1,"[0.054327667, -0.31370223, 0.1041381, -0.46162...",0.625505,0.298504,0.471162,0.612973,1,0
4,airman,0,"[-0.20044291, -0.020562049, -0.276353, -0.0372...","[0.49023438, -0.15917969, -0.022460938, -0.024...",1,"[-0.27850354, -0.06571202, 0.04588692, -0.3983...",0.474893,0.260034,0.424119,0.595767,1,0


In [179]:
from sklearn.decomposition import PCA

# Perform PCA to reduce to 2 dimensions
pca = PCA(n_components=2)

X = gendered_words['selected_tokens'].tolist()
principal_components = pca.fit_transform(X)
print("Explained variance ratio: ", pca.explained_variance_ratio_)
gendered_words['pc1'] = principal_components[:, 0]
gendered_words['pc2'] = principal_components[:, 1]

gendered_words['pc1_norm'] = (gendered_words['pc1'] - gendered_words['pc1'].min()) / (gendered_words['pc1'].max() - gendered_words['pc1'].min())
gendered_words['pc2_norm'] = (gendered_words['pc2'] - gendered_words['pc2'].min()) / (gendered_words['pc2'].max() - gendered_words['pc2'].min())

gendered_words.head()

Explained variance ratio:  [0.17536841 0.07030329]


,word,gender_binary,bert_token,word2vec_token,word_count,selected_tokens,pc1,pc2,pc1_norm,pc2_norm,cluster,svm_cluster
0,abbot,0,"[-0.39571184, -0.093838364, 0.06868138, 0.1723...","[0.40039062, 0.41015625, 0.36523438, 0.2207031...",1,"[-0.29727972, 0.13706006, 0.25577983, -0.35511...",-0.756501,-0.101579,0.155368,0.445199,0,0
1,abbots,0,"[-0.18790531, -0.077521764, -0.3876859, 0.1816...","[0.30664062, 0.33398438, 0.1953125, 0.50390625...",1,"[-0.09587786, 0.34578195, 0.050730024, -0.1052...",0.909409,0.173366,0.573701,0.539008,1,0
2,adulterer,0,"[-0.45352724, -0.38397712, -0.25277817, -0.266...","[0.42773438, -0.28515625, -0.0625, 0.020263672...",1,"[0.026859095, -0.04974738, -0.00580548, -0.274...",0.841506,0.092927,0.556650,0.511563,1,0
3,adulterers,0,"[-0.17249976, -0.15101261, -0.3717648, -0.0279...","[-0.14160156, -0.22558594, -0.042236328, 0.208...",1,"[0.054327667, -0.31370223, 0.1041381, -0.46162...",1.283997,0.267746,0.667765,0.571209,1,0
4,airman,0,"[-0.20044291, -0.020562049, -0.276353, -0.0372...","[0.49023438, -0.15917969, -0.022460938, -0.024...",1,"[-0.27850354, -0.06571202, 0.04588692, -0.3983...",0.504301,-0.504769,0.471973,0.307633,1,0


In [180]:
import plotly.express as px

color_map = {0: '#87CEEB', 1: '#FFC0CB'}

# Create the scatter plot
fig = px.scatter(gendered_words, 
                 x='pc1', 
                 y='pc2', 
                 color='gender_binary',  # Use the 'gender' column to determine colors
                 hover_name='word',
                 title='Interactive Map of Job Titles',
                 labels={'pc1': 'Principal Component 1', 'pc2': 'Principal Component 2'},
                 color_continuous_scale=list(color_map.values()),
                 color_discrete_map=color_map,
                 opacity =0.8)  

# Update layout to improve the appearance
fig.update_layout(title='Interactive Job Titles Map',
                  xaxis_title='Principal Component 1',
                  yaxis_title='Principal Component 2',
                  plot_bgcolor='white',
                  paper_bgcolor='white',
                  hovermode='closest')

fig.update_layout(
    xaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='lightgray'
    ),
    yaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='lightgray'
    )
)

# Show the plot
fig.show()


In [181]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
import plotly.express as px

# Assuming 'gendered_words' DataFrame is already defined with 'pc1', 'pc2', 'gender_binary', 'word'

# Prepare the data
X = gendered_words[['pc1', 'pc2']]
y = gendered_words['gender_binary']

# Train a linear SVM
svm = SVC(kernel='linear', C=1.0, random_state=42)
svm.fit(X, y)

# Predict the class labels
gendered_words['svm_cluster'] = svm.predict(X)

# Define a color map for the SVM clusters
svm_color_map = {0: '#87CEEB', 1: '#FFC0CB'}  # Blue for 0, Pink for 1

# Create the scatter plot for SVM classification
fig = px.scatter(gendered_words, 
                 x='pc1', 
                 y='pc2', 
                 color='svm_cluster',  # Use the 'svm_cluster' column for coloring
                 hover_name='word',
                 title='Linear SVM Classification of Job Titles',
                 labels={'pc1': 'Principal Component 1', 'pc2': 'Principal Component 2'},
                 color_discrete_map=svm_color_map,
                 opacity=0.8)  

# Update layout to improve the appearance
fig.update_layout(title='Linear SVM Classification of Job Titles',
                  xaxis_title='Principal Component 1',
                  yaxis_title='Principal Component 2',
                  plot_bgcolor='white',
                  paper_bgcolor='white',
                  hovermode='closest')

fig.update_layout(
    xaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='lightgray'
    ),
    yaxis=dict(
        gridcolor='lightgray',
        zerolinecolor='lightgray'
    )
)

# Show the plot
fig.show()

print("Accuracy: ", svm.score(X, y))

Accuracy:  0.605568445475638


In [182]:
n = 20
pcs = ['pc1', 'pc2']
top_n_desc_pc1 = gendered_words.nlargest(n, pcs[0]).word.values
top_n_asc_pc1 = gendered_words.nsmallest(n, pcs[0]).word.values
top_n_desc_pc2 = gendered_words.nlargest(n, pcs[1]).word.values
top_n_asc_pc2 = gendered_words.nsmallest(n, pcs[1]).word.values

top_n_desc_pc1_values = gendered_words.nlargest(n, pcs[0])[f'{pcs[0]}'].values
top_n_asc_pc1_values = gendered_words.nsmallest(n, pcs[0])[f'{pcs[0]}'].values
top_n_desc_pc2_values = gendered_words.nlargest(n, pcs[1])[f'{pcs[1]}'].values
top_n_asc_pc2_values = gendered_words.nsmallest(n, pcs[1])[f'{pcs[1]}'].values

table = pd.DataFrame({
    'Top 20 Descending PC1': top_n_desc_pc1,
    'PC1_desc': top_n_desc_pc1_values,
    'Top 20 Ascending PC1': top_n_asc_pc1,
    'PC1_asc': top_n_asc_pc1_values,
    'Top 20 Descending PC2': top_n_desc_pc2,
    'PC2_desc': top_n_desc_pc2_values,
    'Top 20 Ascending PC2': top_n_asc_pc2,
    'PC2_asc': top_n_asc_pc2_values
})

table

,Top 20 Descending PC1,PC1_desc,Top 20 Ascending PC1,PC1_asc,Top 20 Descending PC2,PC2_desc,Top 20 Ascending PC2,PC2_asc
0,seamstresses,2.607042,chick,-1.375216,duchesses,1.524488,servicemen,-1.406411
1,seamstress,2.319096,cowboys,-1.320092,grandmothers,1.443097,men,-1.326466
2,paternity,2.261126,godfather,-1.304902,aunts,1.369928,serviceman,-1.261213
3,congresswomen,2.242336,his,-1.285455,grandmas,1.313999,businessman,-1.258217
4,postmistresses,2.050371,bro,-1.275531,dads,1.277492,councilman,-1.169961
5,spokeswomen,2.019341,grandfather,-1.262878,grandfathers,1.270627,girls,-1.119470
6,headmistresses,1.997588,mister,-1.232653,stepfathers,1.229755,airmen,-1.107607
7,manservant,1.825768,he,-1.209337,brides,1.215071,barman,-1.087642
8,stepdaughter,1.815543,lad,-1.208763,grandpas,1.179283,policeman,-1.086333
9,stepdaughters,1.737715,guy,-1.197178,stepmothers,1.155229,women,-1.046878
